In [4]:
import pandas as pd
import psycopg2

connection = {
    'host': 'dsstudents.skillbox.ru',
    'port': 5432,
    'dbname': 'human_resources',
    'user': 'readonly',
    'password': '6hajV34RTQfmxhS'
}
conn = psycopg2.connect(**connection)

hr_dataset_columns = pd.read_sql_query("""
    SELECT *
    FROM hr_dataset;
    """, conn).columns
hr_dataset = pd.read_sql_query("""
    SELECT *
    FROM hr_dataset;
    """, conn)
production_staff = pd.read_sql_query("""
    SELECT *
    FROM production_staff;
    """, conn)
salary_grid = pd.read_sql_query("""
    SELECT *
    FROM salary_grid;
    """, conn)

print("Есть зависимость между количеством сотрудников и штатом их происхождения")
pd.read_sql_query("""
    select "state", count(*) as total from hr_dataset
    group by "state"
    order by "total" desc
    """, conn)

Есть зависимость между количеством сотрудников и штатом их происхождения


,state,total
0,MA,275
1,CT,6
2,TX,3
3,VT,2
4,NV,1
5,KY,1
6,OH,1
7,NY,1
8,IN,1
9,FL,1


In [5]:
print("Есть зависимость между расой и семейным положением")
pd.read_sql_query("""
    select "maritaldesc", "racedesc", count(*) as total from hr_dataset
    group by "maritaldesc", "racedesc"
    order by "racedesc"
    """, conn)

Есть зависимость между расой и семейным положением


,maritaldesc,racedesc,total
0,Separated,American Indian or Alaska Native,1
1,Divorced,American Indian or Alaska Native,2
2,Married,American Indian or Alaska Native,1
3,Divorced,Asian,3
4,Single,Asian,13
5,Separated,Asian,1
6,Widowed,Asian,2
7,Married,Asian,15
8,Divorced,Black or African American,5
9,Separated,Black or African American,4


In [6]:
print("Есть зависимость между возрастом и Performance Score")
pd.read_sql_query("""
    select "age", "Performance Score", count(*) from hr_dataset
    where "Performance Score" = 'Fully Meets'
    group by "age", "Performance Score"
    order by "age"
    """, conn)

Есть зависимость между возрастом и Performance Score


,age,Performance Score,count
0,25,Fully Meets,2
1,26,Fully Meets,1
2,27,Fully Meets,3
3,28,Fully Meets,5
4,29,Fully Meets,9
5,30,Fully Meets,10
6,31,Fully Meets,9
7,32,Fully Meets,7
8,33,Fully Meets,10
9,34,Fully Meets,11


In [7]:
print("Есть зависимость между отделом и средней зарплатой")
pd.read_sql_query("""
    select s1.*, s2.* from (select distinct "position", "department" from hr_dataset) as s1
    join (select "Position", "Salary Mid" from salary_grid) as s2
    on s1."position" = s2."Position"
    order by "Salary Mid" desc
    """, conn)

Есть зависимость между отделом и средней зарплатой


,position,department,Position,Salary Mid
0,Sr. DBA,IT/IS,Sr. DBA,116007
1,Sr. Network Engineer,IT/IS,Sr. Network Engineer,99458
2,Sr. Accountant,Admin Offices,Sr. Accountant,76988
3,Database Administrator,IT/IS,Database Administrator,68306
4,Network Engineer,IT/IS,Network Engineer,66850
5,Accountant I,Admin Offices,Accountant I,51425
6,Production Technician II,Production,Production Technician II,48000
7,Production Technician I,Production,Production Technician I,40000
8,Administrative Assistant,Admin Offices,Administrative Assistant,40000
